# L5kit dataloader memory usage

In [ ]:
import psutil
def memory(verbose=True):
    mem = psutil.virtual_memory()
    gb = 1024*1024*1024
    if verbose:
        print('Physical memory:',
              '%.2f GB (used),'%((mem.total - mem.available) / gb),
              '%.2f GB (available)'%((mem.available) / gb), '/',
              '%.2f GB'%(mem.total / gb))
    return (mem.total - mem.available) / gb

In [ ]:
memory()

# Environment setup

 - Please add [pestipeti/lyft-l5kit-unofficial-fix](https://www.kaggle.com/pestipeti/lyft-l5kit-unofficial-fix) as utility script.
    - Official utility script "[philculliton/kaggle-l5kit](https://www.kaggle.com/mathurinache/kaggle-l5kit)" does not work with pytorch GPU.

Click "File" botton on top-left, and choose "Add utility script". For the pop-up search window, you need to remove "Your Work" filter, and search [pestipeti/lyft-l5kit-unofficial-fix](https://www.kaggle.com/pestipeti/lyft-l5kit-unofficial-fix) on top-right of the search window. Then you can add the kaggle-l5kit utility script. It is much faster to do this rather than !pip install l5kit every time you run the notebook. 

If successful, you can see "usr/lib/lyft-l5kit-unofficial-fix" is added to the "Data" section of this kernel page on right side of the kernel.

- Also please add [pretrained baseline model](https://www.kaggle.com/huanvo/lyft-pretrained-model-hv)

Click on the button "Add data" in the "Data" section and search for lyft-pretrained-model-hv. If you find the model useful, please upvote it as well.  

In [ ]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
from tqdm import tqdm

import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import matplotlib.pyplot as plt

import os
import random
import time

import warnings
warnings.filterwarnings("ignore")

In [ ]:
l5kit.__version__

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
set_seed(42)

# Configs

In [ ]:
# --- Lyft configs ---
cfg = {
    'format_version': 4,
    'data_path': '/kaggle/input/lyft-motion-prediction-autonomous-vehicles',
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name': "model_resnet34_output",
        'lr': 1e-6,
        'weight_path': '/kaggle/input/lyft-pretrained-model-hv/model_multi_update_lyft_public.pth',
        'train': True,
        'predict': True,
    },
    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
    },
    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 0,
    },    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 0,
    },
    'train_params': {
        'max_num_steps': 4001,
        'checkpoint_every_n_steps': 40,
    }
}

# Load the train and test datasets

In [ ]:
%%time
# set env variable for data
DIR_INPUT = cfg["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager()

In [ ]:
%%time
# Build rasterizer
rasterizer = build_rasterizer(cfg, dm)

In [ ]:
%%time
# Train dataset
train_cfg = cfg["train_data_loader"]
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open(cached=False)
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], 
                              batch_size=train_cfg["batch_size"], num_workers=train_cfg["num_workers"])
print(train_dataset)

In [ ]:
%%time
# Test dataset
test_cfg = cfg["test_data_loader"]
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open(cached=False)
test_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset, shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"], num_workers=test_cfg["num_workers"])
print(test_dataset)

# Load Training data
Next let us implement the training loop, when the **train** parameter is set to True. 

In [ ]:
from IPython.display import display

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
%%time
if cfg["model_params"]["train"]:
    tr_it = iter(train_dataloader)
    progress_bar = tqdm_notebook(range(1, 1 + cfg["train_params"]["max_num_steps"]), mininterval=5)
    num_iter = cfg["train_params"]["max_num_steps"]
    losses_train = []
    iterations = []
    metrics = []
    times = []
    model_name = cfg["model_params"]["model_name"]
    t_start = time.time()
    t_prev = t_start
    memory_used_prev = memory()
    print('== Start loading train data ==')
    for i in progress_bar:
        try:
            data = next(tr_it)
        except StopIteration:
            tr_it = iter(train_dataloader)
            data = next(tr_it)
        
        if i % cfg['train_params']['checkpoint_every_n_steps'] == 0:
            memory_used = memory(False)
            t = time.time()
            iterations.append(i)
            metrics.append(memory_used)
            times.append((t - t_start) / 60)
            print('%5d -'%i, 
                  'dt = %5.3fmins,'%((t - t_prev) / 60), 
                  'dm = %5.3fGB,'%(memory_used - memory_used_prev), 
                  'elapsed_time = %6.2fmins,'%((t - t_start) / 60),
                  'memory_used = %6.3fGB'%memory_used,
                 )
            t_prev = t
            memory_used_prev = memory_used

    results = pd.DataFrame({'iterations': iterations, 'memory (GB)': metrics, 'elapsed_time (mins)': times})
    results.to_csv(f"train_loading_time.csv", index = False)
    print(f"Total training time is {(time.time() - t_start) / 60} mins")
    memory()
    display(results)

In [ ]:
plt.plot(results['elapsed_time (mins)'], results['memory (GB)'])
plt.xlabel('elapsed_time (mins)')
plt.ylabel('memory usage (GB)')
plt.show()

In [ ]:
plt.plot(results['iterations'], results['memory (GB)'])
plt.xlabel('iterations')
plt.ylabel('memory usage (GB)')
plt.show()